In [ ]:
import pandas as pd

pd.options.mode.copy_on_write = True

pd.set_option("display.max_colwidth", None)
pd.set_option("display.max_columns", None)  # show all cols

# url encoding
from urllib.parse import quote

# import requests
from tqdm.auto import tqdm
import numpy as np

import matplotlib.pyplot as plt

plt.style.use("ggplot")
plt.style.use("seaborn-v0_8-colorblind")

from seaborn import kdeplot

from umap import UMAP
from sklearn.manifold import TSNE

In [ ]:
# df = pd.read_csv('../../../dim-bridge-data/fernando-MOF/function_FE_text_data_MOFs_2vis.csv')
# df = pd.read_csv('../../../dim-bridge-data/fernando-MOF/polymorphs_data_DimBridge.csv')


# # general:
# # use both dLM_FE, FE
# # LISAFP and LFS - want to preserve nan points if possible
# # do not use polymorphic_family_ID
# # set(df['synth']) # 10 => 0 , not numerical, dont use
# df = pd.read_csv("GENERAL_data_DimBridge.csv")
# df[df["synth"] == 10] = 0
# df = df.drop(["synth", "polymorphic_family_ID"], axis=1)

# poly:
# use memberX, Family, Parent_MOF_name(categorical)
df = pd.read_csv('POLYMORPHS_data_DimBridge.csv')

# replace nan with random values
for c in df.columns:
    if df[c].isna().sum() > 0:
        print("removing nans from", c)
        values = df.loc[~df[c].isna(), c].to_numpy()
        m = df[c].isna().sum()
        df.loc[df[c].isna(), c] = np.random.choice(values, m)
        
df.head(5)

In [ ]:
# # remove nan rows
# df = df[~df.isna().any(axis=1)]

In [ ]:
# right hand side / predicate

## point similarity for UMAP / tSNE, 'textures'
# ['metal_nonmetal_ratio', 'SE', 'LFS', 'LISAFP', 'SA_vol', 'SA_gra', 'inv_dens', 'VF']

## attributes
# ['fun_MW', 'fun_eps', 'fun_vol', 'fun_elen', 'delta_FE', 'delta_VF', 'FE', 'dLM_FE']

## color by
# ['synth', 'fun_full_name', 'fun_group']

In [ ]:
# umap_attrs = [
#     'metal_nonmetal_ratio', 'SE',
#     'LFS', 'LISAFP',  ## columns that have nans
#     'SA_vol',
#     'SA_gra', 'inv_dens', 'VF',
# ]

# umap_attrs = [
#     "LFS",
#     "LISAFP",
#     "SA_vol",
#     "SA_gra",
#     "inv_dens",
#     "VF",
# ]
# umap_attrs = ['fun_MW', 'fun_eps', 'fun_vol', 'fun_elen', 'fun_sat']


# use all float64 columns in UMAP
umap_attrs = [name for name, dtype in list(df.dtypes.items()) if dtype == np.float64]
print(umap_attrs)

data = df[umap_attrs].to_numpy()

In [ ]:
# normalize
mu = np.array([np.mean(d[~np.isnan(d)]) for d in data.T])
std = np.array([np.std(d[~np.isnan(d)]) for d in data.T])
data = (data - mu) / std

In [ ]:
xy = UMAP(n_neighbors=30, min_dist=0.9).fit_transform(data)
# xy = TSNE().fit_transform(data)

In [ ]:
for column in df.columns:
    if df[column].dtype == np.float64:
        sca = plt.scatter(xy[:,1], xy[:,0], s=2, c=df[column])
        plt.axis('equal')
        plt.title(f'Colored by {column}')
        plt.colorbar()
        plt.show()
        # break

In [ ]:
df['x'] = xy[:,0]
df['y'] = xy[:,1]


In [ ]:
df.to_csv('polymorphs.csv', index=False)

In [ ]:
# plot UMAP colored by 'fun_group'

# color column
# color_column = pd.CategoricalIndex(df['fun_group'], )
color_column = pd.CategoricalIndex(
    df["fun"],
)
color_column.categories

c = color_column.codes  # category to integer code
n_categories = len(color_column.categories)


plt.figure(figsize=[20, 12], dpi=120)
sca = plt.scatter(xy[:, 0], xy[:, 1], s=20, c=c, cmap="tab20", vmin=-0.5, vmax=n_categories - 0.5)
plt.axis("equal")

# colorbar with custom tick labels
cbar = plt.colorbar(sca, ticks=np.arange(n_categories))
cbar.ax.set_yticklabels(color_column.categories)  # vertically oriented colorbar
plt.show()

In [ ]:
# append the umap coordinates and save data as a new csv file

df["x"] = xy[:, 0]
df["y"] = xy[:, 1]

df.to_csv("mof_umap.csv", index=False)

In [ ]:
# plt.figure(figsize=[20,12])
# C = ['#a6cee3','#1f78b4','#b2df8a','#33a02c','#fb9a99','#e31a1c','#fdbf6f','#ff7f00','#cab2d6','#6a3d9a','#ffff99','#b15928', '#333']

# for i, c in enumerate(C):
#     subset = np.array(fun_full_name_index)==i
#     plt.scatter(xy[subset,0], xy[subset,1], s=20, c=c)
# plt.axis('equal');

In [ ]:
# cmap = ['#a6cee3','#1f78b4','#b2df8a','#33a02c','#fb9a99','#e31a1c','#fdbf6f','#ff7f00','#cab2d6','#6a3d9a','#ffff99','#b15928']